### Notebook to compute travel demand with Gravitational Model and travel weights using LOGIT model

#### Initial data

In [4]:
# number of travels passed as a matrix with produced travels on lines and attracted ones on columns
travs = [[40, 110, 150],
         [50, 20, 30],
         [110, 30, 10]]

# the matching friction factors, same arrangement

ffs = [[0.753, 1.597, 0.753],
       [0.987, 0.753, 0.765],
       [1.597, 0.765, 0.753]]

# neutral calibration coefficients

k_ij0 = [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]]

# auto travels cost

tca = [[0.5, 1, 1.4],
       [1.2, 0.8, 1.2],
       [1.7, 1.5, 0.7]]

# transit travels cost

tct = [[1, 1.5, 2],
       [1.8, 1.2, 1.9],
       [1.7, 1.5, 0.7]]

# auto travels duration

tda = [[3, 12, 7],
       [13, 3, 19],
       [9, 16, 4]]

# transit travels duration

tdt = [[15, 5, 12],
       [15, 6, 26],
       [20, 21, 8]]

# the future friction factors

ffs_f = [[0.753, 0.987, 1.597],
         [0.987, 0.753, 0.765],
         [1.597, 0.765, 0.753]]

# the future produced travels

P_is = [750, 580, 480]

# the future attracted travels

A_js = [722, 786, 302]

#### Class to compute the travels using Gravitational Model

In [227]:
class GravitMod:
    def __init__(self, travs, ffs, k_ijs, P_is, A_js):
        self.travs = travs
        self.ffs = ffs
        self.k_ijs = k_ijs
        self.P_is = P_is
        self.A_js = A_js

    
    def transp_mat(mat):
        """
        Method to transpose a squared matrix.
        Takes as input the matrix, returns the transpose, same shape.
        """
        # check for same number of rows and columns (square matrix)
        if(len(mat) != len(mat[0])):
            print("The matrix is not squared. Please provide a squared matrix.")
            exit()

        # transpose the matrix
        transp = list(zip(*mat))

        return [list(sublist) for sublist in transp]

    
    def sum_mat(mat):
        """
        Method to compute the sum of rows of a squared matrix.
        Takes as input the matrix, returns the sum of rows.
        """

        if(len(mat) != len(mat[0])):
            print("The matrix is not squared. Please provide a squared matrix.")
            exit()

        sum_mat = []    # empty list to store the sums

        for row in mat:
            sum_mat.append(sum(row))

        return sum_mat
    
    
    def formula(s_Pi, s_Aj, ffs, k_ijs):
        """
        Method to apply the Gravitational Model formula.
        Takes as inputs the sum of produced, respectively attracted travels,
        the matrix of friction factors, and the matrix of calibration coefficients.
        Returns the squared matrix of computed travels.
        """

        gvals_init = []    # to store computed values
        for i in range(len(travs)):
            pdsum = 0
            for j1, j2 in zip(s_Aj, ffs[i]):
                pdsum = pdsum + j1 * j2
            for k1 in range(len(ffs[i])):
                gvals_init.append((s_Pi[i] * ffs[i][k1] * s_Aj[k1] * k_ijs[i][k1] / pdsum))

        return gvals_init
        
    
    def rnd(mat):
        """
        Method to round the travels raw computed with Gravitational Model.
        Takes as input the single row matrix, returns rounded values, same shape.
        """
        mat_r = []
        for item in mat:
            mat_r.append(round(item))

        return mat_r
    
    def mat_grp(mat):
        """
        Method to group the rounded single row matrix into a squared one.
        Takes as input the single row matrix of rounded travels and
        returns the squared matrix of travels obtained with
        Gravitational Model.
        """
        return [mat[i:i + 3] for i in range(0, len(mat), 3)]
    
    
    def gravmod_init(travs, ffs, k_ijs):
        """
        Method to compute gravitational model values in order to determine the
        calibration factors.
        Takes as input the travels, friction factors and calibration
        coefficients matrices.
        Returns a matrix with the computed travels.
        """
    
        # check if the matrices have the same shape
        if(len(travs) != len(ffs) or (len(travs) != len(k_ijs))):
            print("The matrices doesn't match. Please fix it.")
            exit()
    
        # transpose the travels and friction factors matrices
        travs_t = GravitMod.transp_mat(travs)
        ffs_t = GravitMod.transp_mat(ffs)
    
        # get attracted travels sums
        s_Aj = GravitMod.sum_mat(travs_t)

        # get produced travels sums
        s_Pi = GravitMod.sum_mat(travs)
    
        # compute travels with gravitational model
        gvals_init = GravitMod.formula(s_Pi, s_Aj, ffs, k_ijs)

        # round the computed single row matrix
        gvals_init_r = GravitMod.rnd(gvals_init)
    
        # group flatten list 'gvals_init_r' as a matrix
        gvals_init_m = GravitMod.mat_grp(gvals_init_r)
        
        return gvals_init_m

    
    def gravmod_fin(ffs, k_ijs, P_is, A_js):
        """
        Method to compute future travels using gravitational model.
        Takes as inputs the future friction factors matrix, the previously
        computed calibration coefficients matrix, the matrix of produced
        travels and the matrix of attracted travels.
        Returns a matrix with future travels determined with gravitational
        model.
        """
        
        # check if the matrices have the same shape
        if(len(k_ijs) != len(ffs) or (len(k_ijs) != len(P_is)) or \
           (len(k_ijs) != len(A_js))):
            print("The matrices doesn't match. Please fix it.")
            exit()
    
        # compute travels with gravitational model
        gvals_fin = gvals_init = GravitMod.formula(P_is, A_js, ffs, k_ijs)
    
        # round the number of travels
        gvals_fin_r = GravitMod.rnd(gvals_fin)

        # group flatten list 'gvals_fin_r' as a matrix
        gvals_fin_m = GravitMod.mat_grp(gvals_fin_r)

        return gvals_fin_m

In [213]:
# compute and print travels using Gravitational Model on historical
# data for later calibration coefficients determination
gvalsr = GravitMod.gravmod_init(travs, ffs, k_ij0)
print("Returned from gravmod_init method, ", gvalsr)

Returned from gravmod_init method,  [[82, 140, 78], [43, 26, 31], [82, 31, 37]]


#### Class to adjust the travels computed with Gravitational Model

In [216]:
class IterAdj:
    def __init__(self, travs, travsc, tlr):
        self.travs = travs
        self.travsc = travsc
        self.tlr = tlr

    def comp(s_ih, s_ic, tlr):
        """
        Method to compare two values, within tolerance.
        Takes as inputs the lists of to be compared values
        and the precision/tolerance.
        Returns True or False.
        """
            
        # set a flag
        flag = True

        for ih, ic in zip(s_ih, s_ic):
            if(abs(ih - ic) / ih >= tlr): 
                flag = False
                break

        return flag
    
    def iter_adj_in(travs, travsc, tlr=0.01):
        """
        Method to iteratively adjust travels computed with gravitational model.
        Takes as input the observed (historical) travels,the computed
        ones, in the form of matrices and the precision (tolerance) of
        adjustment.
        Returns a matrix with adjusted travels.
        """
    
        # check if the matrices have the same shape
        if(len(travs) != len(travsc)):
            print("The matrices doesn't match. Please fix it.")
            exit()
        
        # get produced travels sums on observed travels
        s_Pih = GravitMod.sum_mat(travs)
        
        # transpose the observed travels matrix
        Ajh = GravitMod.transp_mat(travs)
        
        # get attracted travels sums on observed travels
        s_Ajh = GravitMod.sum_mat(Ajh)

        cmp_flg = False  # comparison flag to govern the following cycle
        i = 0   # produced passes counter
        j = 0   # attracted passes counter

        while(cmp_flg == False):
                       
            # get produced travels sums on computed travels
            s_Pic = GravitMod.sum_mat(travsc)
            
            if (IterAdj.comp(s_Pih, s_Pic, tlr) == False):
                ccsi = []   # list to store produced travels coefficients
                for ph, pc in zip(s_Pih, s_Pic):
                    ccsi.append(round(ph/pc, 3))

                for x in range(len(travsc)):
                    travsc[x] = [ccsi[x]*item for item in travsc[x]]
            
                i += 1

            # transpose de matrix of computed travels
            Ajc = GravitMod.transp_mat(travsc)

            # get attracted travels sums on computed travels
            s_Ajc = GravitMod.sum_mat(Ajc)
            
            if (IterAdj.comp(s_Ajh, s_Ajc, tlr) == False):
                ccsj = []   # list to store attracted travels coefficients
                for ah, ac in zip(s_Ajh, s_Ajc):
                    ccsj.append(round(ah/ac, 3))

                for x in range(len(Ajc)):
                    Ajc[x] = [ccsj[x]*item for item in Ajc[x]]
            
                j += 1

            travsc = GravitMod.transp_mat(Ajc)    # transpose the transposed

            print("Travels matrix after i = ", i, "and j = ", j, "is ", travsc)
                
            # get attracted travels sums on new computed travels matrix
            # transpose de matrix of computed travels
            Ajc = GravitMod.transp_mat(travsc)

            # get attracted travels sums on computed travels
            s_Ajc = GravitMod.sum_mat(Ajc)

            # update the produced sums
            # get produced travels sums on computed travels
            s_Pic = GravitMod.sum_mat(travsc)
        
            cmp_flg = IterAdj.comp(s_Ajh, s_Ajc, tlr)
                        
            cmp_flg = IterAdj.comp(s_Pih, s_Pic, tlr)
            
            # round the computed travels
            for row in range(len(travsc)):
                travsc[row] = GravitMod.rnd(travsc[row])
        
        return travsc

In [218]:
travsc_adj = IterAdj.iter_adj_in(travs, gvalsr)
print("Historical computed and adjusted, ", travsc_adj)
print("Historical travels matrix, ", travs)

Travels matrix after i =  0 and j =  1 is  [[79.212, 113.68, 101.478], [41.538, 21.112000000000002, 40.330999999999996], [79.212, 25.172, 48.137]]
Travels matrix after i =  1 and j =  1 is  [[80.58, 116.28, 103.02], [40.782, 20.391, 38.839999999999996], [77.973, 24.675, 47.376]]
Historical computed and adjusted,  [[81, 116, 103], [41, 20, 39], [78, 25, 47]]
Historical travels matrix,  [[40, 110, 150], [50, 20, 30], [110, 30, 10]]


#### Function to compute the calibration coefficients

In [221]:
def ccoeffs(travs, travsc):
    """
    Function to compute calibration coefficients for Gravitational Model.
    Takes as inputs computed-adjusted travels, and observed ones.
    Returns a squared matrix with calibration coefficients.
    """
    ccoeffs = []
    for row_h, row_c in zip(travs, travsc):
        for t_h, t_c in zip(row_h, row_c):
            ccoeffs.append(round(t_h / t_c, 2))
    
    ccoeffs_m = GravitMod.mat_grp(ccoeffs)

    return ccoeffs_m

In [223]:
# get the calibration coeffs
cal_coeffs = ccoeffs(travs, travsc_adj)
print("Calibration coefficients matrix, ", cal_coeffs)

Calibration coefficients matrix,  [[0.49, 0.95, 1.46], [1.22, 1.0, 0.77], [1.41, 1.2, 0.21]]


In [229]:
# call the GravitMod future travels method 
# with calibration coefficients and future produced, respectively attracted travels
gravmod_fin = GravitMod.gravmod_fin(ffs_f, cal_coeffs, P_is, A_js)
print("Future travels computed with Gravitational Model and initial data, ", gravmod_fin)

Future travels computed with Gravitational Model and initial data,  [[111, 307, 293], [328, 224, 67], [394, 175, 12]]
